In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
#The Background Dataset
background = pd.read_csv('./datasets/training/background-giants.csv')
background = background.query('PhotoZ > -9999')
background_id = background[['objID', 'ra', 'dec']]

#Dropping the irrelevant features for training SVM model.
background = background.drop(['objID','objID1','ra','dec'], axis=1)
background.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_g,fracDeV_r,fracDeV_i,fracDeV_z,dered_u,dered_g,dered_r,dered_i,dered_z,PhotoZ
0,21.08928,19.37086,18.25864,17.81117,17.51476,21.25800,19.36015,18.24822,17.80149,17.45830,...,1.000000,1.000000,1.000000,1.000000,20.13362,18.32410,17.19614,16.74197,16.40898,0.163294
1,21.41094,19.66853,19.04671,18.49730,18.12082,21.32957,19.64395,18.69971,18.21872,17.81517,...,0.048899,0.111197,0.102860,0.200284,18.94811,17.26758,16.39990,15.94577,15.60334,0.079898
2,21.27979,19.46498,18.33810,17.92343,17.63418,21.25036,19.44884,18.34415,17.88955,17.55409,...,1.000000,1.000000,1.000000,1.000000,20.15441,18.22609,17.11155,16.65807,16.33635,0.158405
3,20.79575,18.89698,17.82768,17.38629,17.21747,20.79277,18.79881,17.79642,17.37555,17.02053,...,0.930281,0.932025,0.997685,1.000000,19.56097,17.58059,16.57519,16.14223,15.82258,0.119085
4,19.69155,17.76127,16.98154,16.55225,16.24577,19.77281,17.82529,16.89516,16.47335,16.13554,...,0.916555,0.941605,0.954829,0.975430,18.35271,16.36183,15.47458,15.03607,14.70395,0.059261


In [3]:
#The Foreground Dataset
foreground = pd.read_csv('./datasets/training/foreground-dwarfs.csv')

foreground_id = foreground[['objID', 'ra', 'dec']]

#Droping the unuseful objID that comes with CrossID
foreground = foreground.drop(['objID','objID1','ra','dec'], axis=1)
foreground.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_g,fracDeV_r,fracDeV_i,fracDeV_z,dered_u,dered_g,dered_r,dered_i,dered_z,PhotoZ
0,20.68459,20.32490,19.90491,19.65816,19.58863,20.93056,19.84648,19.39435,19.13173,18.93974,...,0.000000,0.000000,0.000000,0.024766,17.14250,16.04856,15.63828,15.42416,15.29288,0.025635
1,20.38590,18.79545,18.40734,18.01583,17.85035,20.31682,18.76537,18.05824,17.72758,17.45851,...,0.252313,0.271305,0.262290,0.388589,18.42509,16.79932,16.11763,15.77143,15.49772,0.043808
2,20.37486,19.15429,18.46381,18.16368,18.04567,20.36893,18.84602,18.14171,17.79829,17.57360,...,0.199078,0.165721,0.215467,0.192362,18.37703,16.79943,16.09405,15.74896,15.51344,0.044954
3,20.10566,18.50956,17.90357,17.60673,17.37036,20.09557,18.56411,17.81530,17.47781,17.24816,...,0.726548,0.861712,0.876894,0.829082,17.90406,16.35933,15.61268,15.26653,15.02099,0.059635
4,20.55464,18.93523,18.13289,17.79406,17.84563,20.42518,18.61701,17.89809,17.54489,17.28442,...,0.357629,0.358013,0.392165,0.397820,18.33514,16.66942,15.94401,15.56867,15.34149,0.054202


In [4]:
# Keep concatenated id when for later merge
galaxies_id = pd.concat([foreground_id,background_id])
galaxies_id.head()

,objID,ra,dec
0,1237680311772250464,16.765005,32.389555
1,1237680315521237319,16.792826,32.346872
2,1237680311772250198,16.685466,32.418586
3,1237680315521302864,16.966691,32.256108
4,1237680311772381486,17.066149,32.499011


In [5]:
# In this case the label for background is 0 and foreground will be 1.
background['label'] = 0
foreground['label'] = 1

In [6]:
# The foreground and background sets now have to be concated together and have the label column removed as a seperate 
# array to perform SVM.
galaxies = pd.concat([foreground,background])
galaxies.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_r,fracDeV_i,fracDeV_z,dered_u,dered_g,dered_r,dered_i,dered_z,PhotoZ,label
0,20.68459,20.32490,19.90491,19.65816,19.58863,20.93056,19.84648,19.39435,19.13173,18.93974,...,0.000000,0.000000,0.024766,17.14250,16.04856,15.63828,15.42416,15.29288,0.025635,1
1,20.38590,18.79545,18.40734,18.01583,17.85035,20.31682,18.76537,18.05824,17.72758,17.45851,...,0.271305,0.262290,0.388589,18.42509,16.79932,16.11763,15.77143,15.49772,0.043808,1
2,20.37486,19.15429,18.46381,18.16368,18.04567,20.36893,18.84602,18.14171,17.79829,17.57360,...,0.165721,0.215467,0.192362,18.37703,16.79943,16.09405,15.74896,15.51344,0.044954,1
3,20.10566,18.50956,17.90357,17.60673,17.37036,20.09557,18.56411,17.81530,17.47781,17.24816,...,0.861712,0.876894,0.829082,17.90406,16.35933,15.61268,15.26653,15.02099,0.059635,1
4,20.55464,18.93523,18.13289,17.79406,17.84563,20.42518,18.61701,17.89809,17.54489,17.28442,...,0.358013,0.392165,0.397820,18.33514,16.66942,15.94401,15.56867,15.34149,0.054202,1


**Reducing number of feature based on feature importance:**

In [7]:
# Feature importance gives you a score for each feature of your data, the higher the score more important or relevant
# is the feature towards your output variable.
X = galaxies.drop(['label'], axis=1) # Independent columns
y = galaxies['label'] # Target column, galaxy type
model = ExtraTreesClassifier()
model.fit(X, y)

# Plot graph of feature importances to for better visualization
# feature_importances_ is a inbuilt class
#feat_importances = pd.Series(model.feature_importances_, index=X.columns)
dfscores = pd.DataFrame(model.feature_importances_)
dfcolumns = pd.DataFrame(X.columns)
feat_importances = pd.concat([dfcolumns, dfscores], axis=1)
feat_importances.columns = ['Feature', 'Score']
print(feat_importances.nlargest(20, 'Score'))

          Feature     Score
126     fracDeV_g  0.067681
89       deVRad_z  0.062539
127     fracDeV_r  0.054960
129     fracDeV_z  0.048815
114     expMag_z0  0.042283
95      deVMag_u0  0.039031
128     fracDeV_i  0.032971
16    petroMag_g0  0.032256
120   aperFlux7_u  0.031753
135        PhotoZ  0.030776
112     expMag_r0  0.028534
14   fiber2Mag_z0  0.024703
132       dered_r  0.020190
115  cModelMag_u0  0.019680
97      deVMag_r0  0.019507
17    petroMag_r0  0.019398
6     fiberMag_g0  0.015277
116  cModelMag_g0  0.014717
10   fiber2Mag_u0  0.014202
130       dered_u  0.013519


/Users/danrachou/Documents/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [8]:
# Features from a previous selection with the highest performance of 0.94 (+/- 0.12)
best_features1 = ['deVRad_r', 'fracDeV_i', 'expRad_r', 'fracDeV_z', 'fiberMag_u0', 'dered_g', 'dered_u', 'expMag_r0',
                  'dered_r', 'cModelMag_u0', 'expMag_g0', 'cModelMag_g0', 'fiber2Mag_u0', 'fracDeV_g', 'expMag_u0',
                  'fiber2Mag_i0', 'mE1PSF_g', 'dered_i', 'fiber2Mag_g0', 'psfMag_z0']

# Features from a previous selection with the highest accuracy of 0.93 (+/- 0.13)
best_features2 = ['fracDeV_z', 'fracDeV_i', 'fracDeV_r', 'expMag_u0', 'dered_u', 'fiberMag_u0', 'dered_g',
                  'expMag_i0', 'deVMag_u0', 'petroMag_g0', 'expRad_g', 'psfMag_g0', 'psfMag_z0', 'dered_r',
                  'deVMag_r0', 'expMag_g0', 'fiberMag_g0', 'deVRad_g', 'expRad_z', 'fiber2Mag_z0']

# Features from a previous selection with the highest accuracy of 0.92 (+/- 0.09)
best_features3 = ['aperFlux7_i', 'aperFlux7_r', 'aperFlux7_u', 'aperFlux7_z', 'cModelMag_r0', 'deVMag_u0', 'deVRad_g',
                  'deVRad_i', 'expMag_u0', 'fracDeV_g', 'fracDeV_r', 'fracDeV_u', 'mCr4_g', 'mE1_g', 'mE2_g',
                  'mRrCc_i', 'mRrCc_z', 'petroR50_r', 'petroR50_u', 'petroR90_u']

# Features from a previous selection with the highest accuracy of 0.94 (+/- 0.13)
best_features4 = ['deVRad_g', 'deVMag_u0', 'fracDeV_g', 'expMag_u0', 'fracDeV_u', 'cModelMag_r0', 'mRrCc_i', 
                 'fracDeV_r', 'deVRad_i', 'petroR50_r', 'dered_g', 'cModelMag_i0', 'fiberMag_z0', 'fracDeV_z', 
                 'deVMag_z0', 'fracDeV_i', 'cModelMag_g0', 'expRad_i', 'dered_z', 'dered_u']

# Features from a previous selection with the highest accuracy of 0.94 (+/- 0.07)
best_features5 = ['PhotoZ', 'fracDeV_i', 'fracDeV_z', 'fracDeV_r', 'cModelMag_r0', 'petroMag_z0', 'fracDeV_g', 
                  'deVRad_z', 'dered_g', 'cModelMag_g0', 'dered_r', 'fiber2Mag_i0', 'mE1PSF_g', 'fiber2Mag_g0',
                  'expMag_g0', 'fiberMag_i0', 'fiberMag_u0', 'expMag_u0', 'petroMag_i0', 'aperFlux7_r']

# Features from a previous selection with the highest accuracy of 0.95 (+/- 0.10)
best_features6 = ['fracDeV_r', 'cModelMag_g0', 'expMag_g0', 'fracDeV_z', 'fracDeV_i', 'deVMag_g0', 'expMag_u0', 
                  'deVRad_i', 'deVMag_r0', 'petroMag_g0', 'PhotoZ', 'dered_g', 'psfMag_z0', 'fracDeV_g', 'deVMag_i0',
                  'fiber2Mag_i0', 'petroR90_r', 'expMag_z0', 'fracDeV_u', 'dered_u']

In [9]:
# Parameter tuning using GridSearchCV
def svc_param_selection(X, y):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1, 10]
    nfolds = 5
    param_grid = {'C': Cs, 'gamma': gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [10]:
#X = galaxies[feat_importances.nlargest(20, 'Score')['Feature']]
X = galaxies[best_features6]
y = galaxies['label']

In [11]:
# Cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, y_train.shape

((458, 20), (458,))

In [12]:
X_test.shape, y_test.shape

((115, 20), (115,))

In [13]:
params = svc_param_selection(X, y)
params

{'C': 10, 'gamma': 0.01}

In [14]:
clf = svm.SVC(C=params['C'], gamma=params['gamma'])
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_test, y_test, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.91 (+/- 0.15)


In [15]:
# Using sigmoid kernel
sigmoidModel = svm.SVC(C=params['C'], gamma=params['gamma'], kernel='sigmoid')
sigmoidModel.fit(X_train, y_train)
scores = cross_val_score(sigmoidModel, X_test, y_test, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.66 (+/- 0.02)


In [16]:
# Using polynomial kernel of different degrees
polyModel_deg1 = svm.SVC(C=params['C'], gamma=params['gamma'], kernel='poly', degree=1)
polyModel_deg1.fit(X_train, y_train)
scores = cross_val_score(polyModel_deg1, X_test, y_test, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.94 (+/- 0.16)


In [17]:
polyModel_deg2 = svm.SVC(C=params['C'], gamma=params['gamma'], kernel='poly', degree=2)
polyModel_deg2.fit(X_train, y_train)
scores = cross_val_score(polyModel_deg2, X_test, y_test, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.95 (+/- 0.10)


In [18]:
polyModel_deg3 = svm.SVC(C=params['C'], gamma=params['gamma'], kernel='poly', degree=3)
polyModel_deg3.fit(X_train, y_train)
scores = cross_val_score(polyModel_deg3, X_test, y_test, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.89 (+/- 0.18)


### Find New Dwarfs in the Persues Cluster
After experimenting with the different feature sets selected by the ExtraTreesClassifier on our svm classifier, we have found a feature set referred to as best_features2. Our polynomial model of 1 degree used this feature set and achieved a accuracy score of 0.94 (+/- 0.04). This is one of the highest scoring models we have trained using our dataset. We will use this model to find new dwarfs in the Perseus Cluster.
1. Merge galaxy_id dataframe with galaxy dataframe. 
* Import dataset of Perseus galaxies in the magnitude range of 15.5 to 16.5. 
* Store object IDs and coordinates in a seperate dataset. 
* Feed the ML model the Perseus cluster dataset. 
* Merge all the datasets (ML model output being the third one).
* Compare the result with the known Perseus cluster dwarfs dataset by counting the duplicates. 
* Generate thumbnail search dataset for visual check of the dataset.

In [19]:
perseus = pd.read_csv('./datasets/predict/perseus-cluster.csv')
perseus = perseus.query('PhotoZ > -9999')
# Store IDs of the different galaxies for later merge.
perseus_id = perseus[['objID', 'ra', 'dec']]
perseus_id.head()

,objID,ra,dec
2,1237661056355598609,50.591805,42.014639
3,1237661059574399521,50.067134,41.482628
4,1237661059574596597,50.502232,41.168978
8,1237661059574202971,49.701229,41.789125
10,1237670960021111003,49.470275,40.897244


In [20]:
perseus.shape[0]

121

In [21]:
perseus_data = perseus[best_features6]
perseus_data.head()

,fracDeV_r,cModelMag_g0,expMag_g0,fracDeV_z,fracDeV_i,deVMag_g0,expMag_u0,deVRad_i,deVMag_r0,petroMag_g0,PhotoZ,dered_g,psfMag_z0,fracDeV_g,deVMag_i0,fiber2Mag_i0,petroR90_r,expMag_z0,fracDeV_u,dered_u
2,0.670438,15.94145,15.79753,1.000000,0.000000,16.35851,18.67158,12.320800,15.52024,15.28876,0.193167,16.72017,22.16000,0.307673,15.75832,16.63384,4.690222,22.05869,0.753482,18.45539
3,0.365564,17.05928,17.38705,0.358714,0.353947,16.68841,18.86445,11.637130,15.90038,17.17827,0.056054,16.63979,18.97378,0.390230,15.56492,19.25706,12.473240,16.14903,0.549013,18.44087
4,0.408501,17.42138,17.65521,0.340118,0.427395,17.09781,18.99165,8.205761,16.39707,17.43354,0.065136,17.12341,18.60661,0.358173,16.01861,18.94808,9.315663,16.35247,0.618591,18.68079
8,0.000000,17.01723,17.01723,0.000000,0.000000,17.07850,23.75492,29.676220,16.24317,17.38794,0.127801,17.06843,20.14462,0.000000,15.20895,20.44827,18.305420,15.05615,0.000000,19.97501
10,0.270072,16.38576,16.52944,0.474151,0.286627,16.04398,18.39639,11.231860,15.31130,16.44180,0.040883,16.55130,17.61852,0.250951,14.92901,18.34813,10.474150,15.15660,0.173596,18.35082


In [22]:
results = polyModel_deg2.predict(perseus_data)
print('Number of objects classified as foreground dwarfs : {}'.format(np.count_nonzero(results == 1)))

Number of objects classified as foreground dwarfs : 84


In [23]:
perseus_id['label'] = results
perseus_id.head()

/Users/danrachou/Documents/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,objID,ra,dec,label
2,1237661056355598609,50.591805,42.014639,1
3,1237661059574399521,50.067134,41.482628,1
4,1237661059574596597,50.502232,41.168978,1
8,1237661059574202971,49.701229,41.789125,1
10,1237670960021111003,49.470275,40.897244,1


In [24]:
ml_dwarfs = perseus_id.loc[perseus_id['label'] == 1]
ml_dwarfs['list'] = 'ml'
ml_dwarfs = ml_dwarfs.drop(['label'], axis=1)
ml_dwarfs.head()

/Users/danrachou/Documents/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,objID,ra,dec,list
2,1237661056355598609,50.591805,42.014639,ml
3,1237661059574399521,50.067134,41.482628,ml
4,1237661059574596597,50.502232,41.168978,ml
8,1237661059574202971,49.701229,41.789125,ml
10,1237670960021111003,49.470275,40.897244,ml


In [25]:
known_dwarfs = pd.read_csv('./datasets/complete/perseus-dwarfs.csv')
known_dwarfs = known_dwarfs[['objID', 'ra', 'dec']]
known_dwarfs['list'] = 'known'
known_dwarfs.head()

,objID,ra,dec,list
0,1237661059574399032,50.021608,41.508698,known
1,1237661059574399032,50.021608,41.508698,known
2,1237661059574399003,49.996059,41.479603,known
3,1237661059574334072,49.932572,41.456982,known
4,1237661059574334104,49.935962,41.446462,known


In [26]:
#known_dwarfs = known_dwarfs.drop_duplicates(subset='objID')
known_dwarfs.shape[0] # Number of known dwarfs from Perseus cluster

183

In [27]:
perseus_dwarfs = pd.concat([known_dwarfs, ml_dwarfs], axis=0)
perseus_dwarfs.shape[0]

267

In [28]:
deduplicated = perseus_dwarfs.drop_duplicates(subset=['objID'], keep=False)
deduplicated.shape[0]

228

In [29]:
new_dwarfs = deduplicated.loc[deduplicated['list'] == 'ml']
new_dwarfs.shape[0]

68

In [30]:
new_dwarfs[['objID', 'ra', 'dec']].to_csv(r'./datasets/thumbnail-search/new-dwarfs.csv', sep=' ', index=False)

### Background Giant Classified as Foreground Dwarfs
We want to see if there are any false positives in our result by searching through a dataset of known background galaxies of the Perseus cluster and see if any on those galaxies were classified as foreground dwarfs in our result set.

In [31]:
perseus_back = pd.read_csv('./datasets/complete/perseus-back.csv')
perseus_back = perseus_back[['objID', 'ra', 'dec']]
perseus_back['list'] = 'known'
perseus_back.head()

,objID,ra,dec,list
0,1237661059574334542,49.863859,41.557747,known
1,1237661122388033863,49.936755,41.398687,known
2,1237661059574334251,49.943267,41.626093,known
3,1237661059574269819,49.789214,41.525355,known
4,1237661059574270262,49.787360,41.545026,known


In [32]:
perseus_back = perseus_back.drop_duplicates(subset=['objID'])
perseus_back.shape[0]

53

In [33]:
mergedlist = pd.concat([perseus_back, ml_dwarfs], axis=0)
mergedlist.shape[0]

137

In [34]:
duplicates = mergedlist[mergedlist.duplicated(subset=['objID'])]
duplicates

,objID,ra,dec,list
24,1237661055281856551,49.883191,41.304234,ml
28,1237661083199144802,49.334194,41.330029,ml
135,1237661055818401198,49.590876,42.076997,ml
